Getting Started

In [9]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional

# This enables response_model keyword
# from client.chat.completions.create
client = instructor.patch(OpenAI())

class Age(BaseModel):
    chain_of_thought: str = Field(...,
        description="Think step by step to determine the correct number of years old the user is.")
    years: int

class UserDetail(BaseModel):
    """This is information about a user.  
    """
    name: str = Field(description="The first and last name of the user.")
    age: Age 
    occupation: Optional[str] = Field(description="The name of the users job.")

user = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "The first person to walk in the room is named Steve Farmpt. He is the same age as the comedian Mel Brooks. He's from the Cornell family. He works down at the shoe factory on the assembly line. Because he's big, people typically address him as Mr. Gantenbien."},
    ]
)



In [10]:
user

UserDetail(name='Steve Farmpt', age=Age(chain_of_thought='the same age as Mel Brooks', years=94), occupation='Assembly Line Worker at the Shoe Factory')

In [28]:
assert isinstance(user, UserDetail)
assert user.name == "Steve Farmpt"
assert user.age == 97
assert user.occupation == "Assembly Line Worker"

AssertionError: 